# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citiesdf=pd.read_csv('output_data/cities.csv')
citiesdf.head()

,City,Country,Date,Humidity,Cloudiness,Lat,Lng,Max Temp,Wind Speed
0,greenville,US,2020-01-26 09:07:19,56.0,1.0,35.61,-77.37,48.20,6.93
1,adrar,MR,2020-01-26 09:07:19,13.0,96.0,20.50,-10.07,86.52,12.28
2,ushuaia,AR,2020-01-26 09:05:55,81.0,75.0,-54.80,-68.30,46.40,24.16
3,port elizabeth,ZA,2020-01-26 09:05:17,63.0,91.0,-33.92,25.57,66.20,34.45
4,busselton,AU,2020-01-26 09:07:20,37.0,0.0,-33.65,115.33,73.99,12.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = citiesdf[["Lat", "Lng"]] 
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout, map_type='TERRAIN')
heat_layer = gmaps.heatmap_layer(locations, weights=citiesdf['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
vacay_df=citiesdf[(citiesdf['Max Temp'] <= 85) & (citiesdf['Max Temp'] >= 70) & (citiesdf['Cloudiness'] < 30) &
                 (citiesdf['Humidity'] < 50) & (citiesdf['Wind Speed'] < 10)]
vacay_df.dropna(how='any')
hotel_df=vacay_df[['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name']=''
hotel_df

C:\Users\ktauf\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,City,Country,Lat,Lng,Hotel Name
24,aswan,EG,24.09,32.91,
164,itapetininga,BR,-23.59,-48.05,
174,diego de almagro,CL,-26.37,-70.05,
181,umarkot,PK,25.36,69.74,
209,buin,CL,-33.73,-70.75,
211,castro,BR,-24.79,-50.01,
364,kharakvasla,IN,18.43,73.77,
443,vallenar,CL,-28.57,-70.76,
499,agadez,NE,19.75,10.25,
501,mecca,SA,21.43,39.83,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
radius = 5000
type = "hotel"
params = {"radius": radius, "type": type, "key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params['location'] = (f"{row['Lat']},{row['Lng']}")
    response = requests.get(base_url, params=params).json()    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        next
hotel_df.iloc[1,1]='NA'
hotel_df

C:\Users\ktauf\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
24,aswan,EG,24.09,32.91,Aswan
164,itapetininga,NA,-23.59,-48.05,Itapetininga
174,diego de almagro,CL,-26.37,-70.05,Diego de Almagro
181,umarkot,PK,25.36,69.74,Umarkot
209,buin,CL,-33.73,-70.75,San Bernardo
211,castro,BR,-24.79,-50.01,Castro
364,kharakvasla,IN,18.43,73.77,Pune
443,vallenar,CL,-28.57,-70.76,Vallenar
499,agadez,NE,19.75,10.25,
501,mecca,SA,21.43,39.83,Mecca


In [13]:
hotel_df=hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df=hotel_df.dropna(how='any')
hotel_df

,City,Country,Lat,Lng,Hotel Name
24,aswan,EG,24.09,32.91,Aswan
164,itapetininga,NA,-23.59,-48.05,Itapetininga
174,diego de almagro,CL,-26.37,-70.05,Diego de Almagro
181,umarkot,PK,25.36,69.74,Umarkot
209,buin,CL,-33.73,-70.75,San Bernardo
211,castro,BR,-24.79,-50.01,Castro
364,kharakvasla,IN,18.43,73.77,Pune
443,vallenar,CL,-28.57,-70.76,Vallenar
501,mecca,SA,21.43,39.83,Mecca


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
marker_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', 
                                  scale=3, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…